## Feedback correction


In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
#
# SPDX-License-Identifier: Apache-2.0
# Copyright 2023 Inria

"""Tiny example: balancing using a proportional wheel controller."""

import gymnasium as gym

import upkie.envs

upkie.envs.register()

if __name__ == "__main__":
    with gym.make("UpkieGroundVelocity-v3", frequency=200.0) as env:
        observation, _ = env.reset()  # connects to the spine
        action = 0.0 * env.action_space.sample()
        for step in range(10000):
            pitch = observation[0]
            ground_position = observation[1]
            ground_velocity = observation[3]
            v = 10.0 * pitch + 1.0 * ground_position + 0.1 * ground_velocity
            action[0] = v  # action is the next commanded ground velocity
            observation, reward, terminated, truncated, _ = env.step(action)
            env.unwrapped.log("pitch", pitch)
            if terminated or truncated:
                observation, _ = env.reset()


/home/kevin/miniconda3/envs/robotics_course/lib/python3.10/site-packages/gymnasium/envs/registration.py:694: UserWarning: WARN: Overriding environment UpkieGroundVelocity-v3 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/home/kevin/miniconda3/envs/robotics_course/lib/python3.10/site-packages/gymnasium/envs/registration.py:694: UserWarning: WARN: Overriding environment UpkieServoPositions-v4 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/home/kevin/miniconda3/envs/robotics_course/lib/python3.10/site-packages/gymnasium/envs/registration.py:694: UserWarning: WARN: Overriding environment UpkieServoTorques-v4 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/home/kevin/miniconda3/envs/robotics_course/lib/python3.10/site-packages/gymnasium/envs/registration.py:694: UserWarning: WARN: Overriding environment UpkieServos-v4 already in registry.
  

KeyboardInterrupt: 

## Apply sagittal force

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
#
# SPDX-License-Identifier: Apache-2.0
# Copyright 2024 Inria

"""Lift the simulated robot while it balances in place."""

import gymnasium as gym
import numpy as np
import time

import upkie.envs

upkie.envs.register()


def get_vertical_force(
    step: int,
    start: int = 200,
    lift_steps: int = 200,
    delta: float = 0.1,
    hold_steps: int = 400,
) -> float:
    lift: float = 0.0  # 0 = no force, 1 => apply -mg
    if step < start:
        lift = 0.0
    elif step - start < lift_steps // 2:
        lift = 1.0 + delta
    elif step - start < lift_steps:
        lift = 1.0 - delta
    elif step - start - lift_steps < hold_steps:
        lift = 1.0
    else:
        lift = 0.0
    mass = 5.34  # approximative, in [kg]
    return lift * mass * 9.81  # in [N]

def get_sagittal_force(intensity: float, duration=None):

    mass = 5.34 #mass in [kg]
    force  = intensity*mass*9.81 #Force in [N]
    if duration is not None and duration > 1e9:
        force = 0.0
    return force
    
    


def run(env: upkie.envs.UpkieGroundVelocity):
    torso_force_in_world = np.zeros(3)
    bullet_action = {
        "external_forces": {
            "torso": {
                "force": torso_force_in_world,
                "local": False,
            }
        }
    }
    observation, _ = env.reset()
    gain = np.array([10.0, 1.0, 0.0, 0.1])

    #Values of intensity sagitall force that is apply 
    intensity_force = np.arange(0.0,2.0,0.01)
    isFall = np.zeros((intensity_force.shape),dtype=bool)

    torso_force_in_world[0] = get_sagittal_force(intensity_force[0])
    index_intensity = 0
    start = time.time_ns()
    
    for step in range(100000):
        action = gain.dot(observation).reshape((1,))
        now = time.time_ns()
        duration = now-start
        torso_force_in_world[0] = get_sagittal_force(intensity_force[index_intensity],duration = duration)
        env.bullet_extra(bullet_action)  # call before env.step
        observation, _, terminated, truncated, _ = env.step(action)
        if terminated or truncated or duration>3e9:
            if terminated or truncated:
                isFall[index_intensity] = True
            else:
                isFall[index_intensity] = False
            start = time.time_ns()
            if index_intensity < intensity_force.shape[0]:
                index_intensity += 1
            torso_force_in_world[0] = get_sagittal_force(intensity_force[index_intensity])
            print(f'Monitoring : intensity:{intensity_force[index_intensity]}')
            observation, _ = env.reset()
    return intensity_force, isFall


if __name__ == "__main__":
    with gym.make("UpkieGroundVelocity-v3", frequency=200.0) as env:
        intensity_force, isFall = run(env)


/home/kevin/miniconda3/envs/robotics_course/lib/python3.10/site-packages/gymnasium/envs/registration.py:694: UserWarning: WARN: Overriding environment UpkieGroundVelocity-v3 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/home/kevin/miniconda3/envs/robotics_course/lib/python3.10/site-packages/gymnasium/envs/registration.py:694: UserWarning: WARN: Overriding environment UpkieServoPositions-v4 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/home/kevin/miniconda3/envs/robotics_course/lib/python3.10/site-packages/gymnasium/envs/registration.py:694: UserWarning: WARN: Overriding environment UpkieServoTorques-v4 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/home/kevin/miniconda3/envs/robotics_course/lib/python3.10/site-packages/gymnasium/envs/registration.py:694: UserWarning: WARN: Overriding environment UpkieServos-v4 already in registry.
  

Monitoring : intensity:0.01


[2024-11-18 23:09:44,771] [warning] UpkieGroundVelocity rate limiter is late by -1.300000 [ms] (rate_limiter.py:106)


Monitoring : intensity:0.02


[2024-11-18 23:09:47,778] [warning] UpkieGroundVelocity rate limiter is late by -1.600000 [ms] (rate_limiter.py:106)


...

[2024-11-18 23:10:20,670] [warning] UpkieGroundVelocity rate limiter is late by -0.900000 [ms] (rate_limiter.py:106)
[2024-11-18 23:10:20,867] [warning] UpkieGroundVelocity rate limiter is late by -0.600000 [ms] (rate_limiter.py:106)
[2024-11-18 23:10:20,983] [warning] UpkieGroundVelocity rate limiter is late by -0.800000 [ms] (rate_limiter.py:106)
[2024-11-18 23:10:21,022] [warning] UpkieGroundVelocity rate limiter is late by -0.900000 [ms] (rate_limiter.py:106)
[2024-11-18 23:10:21,036] [warning] UpkieGroundVelocity rate limiter is late by -1.000000 [ms] (rate_limiter.py:106)
[2024-11-18 23:10:21,117] [warning] UpkieGroundVelocity rate limiter is late by -1.300000 [ms] (rate_limiter.py:106)
[2024-11-18 23:10:21,367] [warning] ground_velocity=1.0017185098047956 clamped to upper=1.0 (clamp.py:57)
[2024-11-18 23:10:21,373] [warning] UpkieGroundVelocity rate limiter is late by -1.400000 [ms] (rate_limiter.py:106)
[2024-11-18 23:10:21,381] [warning] UpkieGroundVelocity rate limiter is lat

Monitoring : intensity:0.15


[2024-11-18 23:10:23,361] [warning] UpkieGroundVelocity rate limiter is late by -0.800000 [ms] (rate_limiter.py:106)
[2024-11-18 23:10:23,398] [warning] UpkieGroundVelocity rate limiter is late by -1.100000 [ms] (rate_limiter.py:106)
[2024-11-18 23:10:23,417] [warning] UpkieGroundVelocity rate limiter is late by -2.100000 [ms] (rate_limiter.py:106)
[2024-11-18 23:10:23,430] [warning] UpkieGroundVelocity rate limiter is late by -0.600000 [ms] (rate_limiter.py:106)
[2024-11-18 23:10:23,450] [warning] UpkieGroundVelocity rate limiter is late by -1.600000 [ms] (rate_limiter.py:106)
[2024-11-18 23:10:23,463] [warning] UpkieGroundVelocity rate limiter is late by -1.500000 [ms] (rate_limiter.py:106)
[2024-11-18 23:10:23,498] [warning] UpkieGroundVelocity rate limiter is late by -1.600000 [ms] (rate_limiter.py:106)
[2024-11-18 23:10:23,547] [warning] UpkieGroundVelocity rate limiter is late by -0.500000 [ms] (rate_limiter.py:106)
[2024-11-18 23:10:23,683] [warning] UpkieGroundVelocity rate lim

Monitoring : intensity:0.16


[2024-11-18 23:10:25,765] [warning] UpkieGroundVelocity rate limiter is late by -0.800000 [ms] (rate_limiter.py:106)
[2024-11-18 23:10:25,835] [warning] UpkieGroundVelocity rate limiter is late by -1.500000 [ms] (rate_limiter.py:106)
[2024-11-18 23:10:25,844] [warning] UpkieGroundVelocity rate limiter is late by -0.700000 [ms] (rate_limiter.py:106)
[2024-11-18 23:10:25,915] [warning] UpkieGroundVelocity rate limiter is late by -0.600000 [ms] (rate_limiter.py:106)
[2024-11-18 23:10:25,969] [warning] UpkieGroundVelocity rate limiter is late by -1.000000 [ms] (rate_limiter.py:106)
[2024-11-18 23:10:25,978] [warning] UpkieGroundVelocity rate limiter is late by -0.800000 [ms] (rate_limiter.py:106)
[2024-11-18 23:10:25,987] [warning] UpkieGroundVelocity rate limiter is late by -0.900000 [ms] (rate_limiter.py:106)
[2024-11-18 23:10:25,999] [warning] UpkieGroundVelocity rate limiter is late by -0.900000 [ms] (rate_limiter.py:106)
[2024-11-18 23:10:26,151] [warning] UpkieGroundVelocity rate lim

## Apply Sagittal force upgrade

In [10]:
#### !/usr/bin/env python3
# -*- coding: utf-8 -*-
#
# SPDX-License-Identifier: Apache-2.0
# Copyright 2024 Inria

"""Lift the simulated robot while it balances in place."""

import gymnasium as gym
import numpy as np
import time

import upkie.envs

upkie.envs.register()

#Vérifier si le calcul d'intensité est bon
def get_sagittal_force(intensity: float, duration=None):

    mass = 5.34 #mass in [kg]
    force  = intensity*mass*9.81 #Force in [N]
    if duration is not None and duration > 1e9:
        force = 0.0
    return force
    
    


def run(env: upkie.envs.UpkieGroundVelocity):
    torso_force_in_world = np.zeros(3)
    bullet_action = {
        "external_forces": {
            "torso": {
                "force": torso_force_in_world,
                "local": False,
            }
        }
    }
    observation, _ = env.reset()
    gain = np.array([10.0, 1.0, 0.0, 0.1])

    #Values of intensity sagitall force that is apply 
    intensity_force = np.arange(0.0,0.1,0.01)
    isFall = False

    magnitude = get_sagittal_force(intensity_force[0])
    previous_magnitude = magnitude
    
    torso_force_in_world[0] = magnitude
    
    best_magnitude = 0.0
    
    index_intensity = 0
    start = time.time_ns()
    
    while isFall==False and index_intensity < intensity_force.shape[0]:
        
        action = gain.dot(observation).reshape((1,))
        
        now = time.time_ns()
        duration = now-start
        magnitude = get_sagittal_force(intensity_force[index_intensity],duration = duration)
        
        torso_force_in_world[0] = magnitude
        env.bullet_extra(bullet_action)  # call before env.step
        observation, _, terminated, truncated, _ = env.step(action)
        
        if terminated or truncated or duration>3e9:
            if terminated or truncated:
                print('est tombé')
                isFall = True
                best_magnitude = previous_magnitude
            start = time.time_ns()
            if index_intensity < intensity_force.shape[0]:
                index_intensity += 1
            previous_magnitude = magnitude
            magnitude = get_sagittal_force(intensity_force[0])
            torso_force_in_world[0] = get_sagittal_force(intensity_force[index_intensity])
            print(f'Monitoring : intensity:{intensity_force[index_intensity]}')
            observation, _ = env.reset()
    return best_magnitude


if __name__ == "__main__":
    with gym.make("UpkieGroundVelocity-v3", frequency=200.0) as env:
        best_magnitude2 = run(env)
        print (best_magnitude2)


[2024-11-19 12:08:25,532] [warning] UpkieGroundVelocity rate limiter is late by -2.400000 [ms] (rate_limiter.py:106)
[2024-11-19 12:08:25,533] [warning] ground_velocity=-15.27310013764141 clamped to lower=-1.0 (clamp.py:54)
[2024-11-19 12:08:25,589] [warning] UpkieGroundVelocity rate limiter is late by -9.900000 [ms] (rate_limiter.py:106)
[2024-11-19 12:08:25,636] [warning] UpkieGroundVelocity rate limiter is late by -6.200000 [ms] (rate_limiter.py:106)
[2024-11-19 12:08:25,669] [warning] UpkieGroundVelocity rate limiter is late by -4.200000 [ms] (rate_limiter.py:106)
[2024-11-19 12:08:28,545] [warning] UpkieGroundVelocity rate limiter is late by -1.000000 [ms] (rate_limiter.py:106)


Monitoring : intensity:0.01


[2024-11-19 12:08:31,554] [warning] UpkieGroundVelocity rate limiter is late by -1.100000 [ms] (rate_limiter.py:106)


Monitoring : intensity:0.02


[2024-11-19 12:08:34,561] [warning] UpkieGroundVelocity rate limiter is late by -1.100000 [ms] (rate_limiter.py:106)


...

Monitoring : intensity:0.05


[2024-11-19 12:08:43,586] [warning] UpkieGroundVelocity rate limiter is late by -1.500000 [ms] (rate_limiter.py:106)
[2024-11-19 12:08:43,587] [warning] ground_velocity=-1.3691883428365834 clamped to lower=-1.0 (clamp.py:54)


Monitoring : intensity:0.06


[2024-11-19 12:08:44,577] [warning] ground_velocity=1.0011042934064134 clamped to upper=1.0 (clamp.py:57)
[2024-11-19 12:08:44,587] [warning] ground_velocity=1.0015966320170733 clamped to upper=1.0 (clamp.py:57)
[2024-11-19 12:08:44,592] [warning] ground_velocity=1.0084954434832194 clamped to upper=1.0 (clamp.py:57)
[2024-11-19 12:08:44,598] [warning] ground_velocity=1.014947452774799 clamped to upper=1.0 (clamp.py:57)
[2024-11-19 12:08:44,603] [warning] ground_velocity=1.0068899690475772 clamped to upper=1.0 (clamp.py:57)
[2024-11-19 12:08:44,608] [warning] ground_velocity=1.0203846401186398 clamped to upper=1.0 (clamp.py:57)
[2024-11-19 12:08:44,613] [warning] ground_velocity=1.017176617054972 clamped to upper=1.0 (clamp.py:57)
[2024-11-19 12:08:44,618] [warning] ground_velocity=1.0231714589333867 clamped to upper=1.0 (clamp.py:57)
[2024-11-19 12:08:44,623] [warning] ground_velocity=1.026585530412694 clamped to upper=1.0 (clamp.py:57)
[2024-11-19 12:08:44,628] [warning] ground_veloci

est tombé
Monitoring : intensity:0.07


In [9]:
print(best_magnitude2)

0.0
